# Fachpraktikum AI University Stuttgart
Author: Siyu Chen


## Install dependencies

In [1]:
!pip install --quiet lineflow
!pip install --quiet transformers
!pip install --quiet pytorch-lightning

     |████████████████████████████████| 2.1MB 9.3MB/s 
     |████████████████████████████████| 3.3MB 18.8MB/s 
     |████████████████████████████████| 901kB 50.4MB/s 
     |████████████████████████████████| 849kB 8.5MB/s 
     |████████████████████████████████| 829kB 27.5MB/s 
     |████████████████████████████████| 276kB 38.3MB/s 
     |████████████████████████████████| 112kB 44.6MB/s 
     |████████████████████████████████| 184kB 36.2MB/s 
     |████████████████████████████████| 1.3MB 37.3MB/s 
     |████████████████████████████████| 296kB 41.1MB/s 
     |████████████████████████████████| 143kB 53.5MB/s 


## Import libraries which are needed for fine-tuning

In [2]:
from typing import Dict
from pathlib import Path
import json
from functools import partial
from collections import OrderedDict
from argparse import ArgumentParser

import lineflow as lf
from transformers import BertForMultipleChoice, BertTokenizer, AdamW
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping

import torch
from torch.utils.data import DataLoader, SequentialSampler, RandomSampler

## Dataset

### Useful function to process dataset

In [19]:
MAX_LEN = 128
NUM_LABELS = 4
label_map = {"A": 0, "B": 1, "C": 2, "D": 3}


def raw_samples_to_dataset(samples):
    datas = []
    for sample in samples:
        for idx in range(len(sample["answers"])):
            _id = sample["id"]
            _article = sample["article"]
            _answer = sample["answers"][idx]
            _options = sample["options"][idx]
            _question = sample["questions"][idx]

            data = {
                    "id": _id,
                    "article": _article,
                    "answer": _answer,
                    "options": _options,
                    "question": _question,
                    }
            datas.append(data)
            
    return lf.Dataset(datas)


def preprocess(tokenizer: BertTokenizer, x: Dict) -> Dict:

    choices_features = []

    option: str
    for option in x["options"]:
        text_a = x["article"]
        if x["question"].find("_") != -1:
            text_b = x["question"].replace("_", option)
        else:
            text_b = x["question"] + " " + option

        inputs = tokenizer.encode_plus(
                text_a,
                text_b,
                add_special_tokens=True,
                max_length=MAX_LEN
                )
        input_ids, token_type_ids = inputs["input_ids"], inputs["token_type_ids"]
        attention_mask = [1] * len(input_ids)

        pad_token_id = tokenizer.pad_token_id
        padding_length = MAX_LEN - len(input_ids)
        input_ids = input_ids + ([pad_token_id] * padding_length)
        attention_mask = attention_mask + ([0] * padding_length)
        token_type_ids = token_type_ids + ([pad_token_id] * padding_length)

        assert len(input_ids) == MAX_LEN, "Error with input length {} vs {}".format(len(input_ids), MAX_LEN)
        assert len(attention_mask) == MAX_LEN, "Error with input length {} vs {}".format(len(attention_mask), MAX_LEN)
        assert len(token_type_ids) == MAX_LEN, "Error with input length {} vs {}".format(len(token_type_ids), MAX_LEN)

        choices_features.append({
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "token_type_ids": token_type_ids,
            })

    labels = label_map.get(x["answer"], -1)
    label = torch.tensor(labels).long()

    return {
            "id": x["id"],
            "label": label,
            "input_ids": torch.tensor([cf["input_ids"] for cf in choices_features]),
            "attention_mask": torch.tensor([cf["attention_mask"] for cf in choices_features]),
            "token_type_ids": torch.tensor([cf["token_type_ids"] for cf in choices_features]),
            }


def get_dataloader(datadir: str, cachedir: str = "./"):
    datadir = Path(datadir)
    cachedir = Path(cachedir)
    batch_size = 8

    tokenizer = BertTokenizer.from_pretrained("bert-base-uncased", do_lower_case=True)
    preprocessor = partial(preprocess, tokenizer)

    train_samples = []
    for grade in ("middle", "high"):
        for _path in (datadir / "train" / grade).iterdir():
            train_samples.append(json.loads(_path.read_text()))
    train = raw_samples_to_dataset(train_samples)
    train_dataloader = DataLoader(
            train.map(preprocessor).save(cachedir / "train.cache"),
            sampler=RandomSampler(train),
            batch_size=batch_size
            )

    val_samples = []
    for grade in ("middle", "high"):
        for _path in (datadir / "dev" / grade).iterdir():
            val_samples.append(json.loads(_path.read_text()))
    val = raw_samples_to_dataset(val_samples)
    val_dataloader = DataLoader(
            val.map(preprocessor).save(cachedir / "val.cache"),
            sampler=SequentialSampler(val),
            batch_size=batch_size
            )

    test_samples = []
    for grade in ("middle", "high"):
        for _path in (datadir / "test" / grade).iterdir():
            test_samples.append(json.loads(_path.read_text()))
    test = raw_samples_to_dataset(test_samples)
    test_dataloader = DataLoader(
            test.map(preprocessor).save(cachedir / "test.cache"),
            sampler=SequentialSampler(test),
            batch_size=batch_size
            )

    return train_dataloader, val_dataloader, test_dataloader

### Method 1: Download dataset and unzip dataset

#### Download dataset

In [8]:
!ls 
!wget http://www.cs.cmu.edu/~glai1/data/race/RACE.tar.gz

drive  sample_data
--2021-05-02 13:20:36--  http://www.cs.cmu.edu/~glai1/data/race/RACE.tar.gz
Resolving www.cs.cmu.edu (www.cs.cmu.edu)... 128.2.42.95
Connecting to www.cs.cmu.edu (www.cs.cmu.edu)|128.2.42.95|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 25443609 (24M) [application/x-gzip]
Saving to: ‘RACE.tar.gz’

RACE.tar.gz         100%[===================>]  24.26M  1.30MB/s    in 50s     

2021-05-02 13:21:26 (497 KB/s) - ‘RACE.tar.gz’ saved [25443609/25443609]



#### Unzip dataset

In [13]:
!ls 
!tar -xf RACE.tar.gz && ls 

drive  RACE.tar.gz  sample_data
drive  RACE  RACE.tar.gz  sample_data


In [6]:
!cd RACE && ls && pwd

dev  test  train
/content/RACE


#### Call functions to handle raw dataset (if the function get_dataloader('/content/RACE') is never called)

In [7]:
train_dataloader, val_dataloader, test_dataloader = get_dataloader('/content/RACE')

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.



Saving data to val_small.cache...
Saving data to test_small.cache...


In [8]:
!ls

RACE  RACE.tar.gz  sample_data	test_small.cache  val_small.cache


#### Copy `train.cache`, `val.cache` and `test.cache` into Google Drive

In [ ]:
!ls
!cp train.cache /content/drive/My\ Drive/Fachpraktikum/
!cp val.cache /content/drive/My\ Drive/Fachpraktikum/
!cp test.cache /content/drive/My\ Drive/Fachpraktikum/

#### Check whether `train.cache`, `val.cache` and `test.cache` are saved

In [ ]:
!ls

### Method 2: Load `DataLoader` from the path which e.g. `train.cache`, `val.cache` and `test.cache`. Only when the function get_dataloader('/content/RACE') is already called and saved into google drive

#### Mounting Google Drive locally

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#### Check whether `train.cache`, `val.cache` and `test.cache` are in Google Drive

In [11]:
!ls /content/drive/My\ Drive/Fachpraktikum/

bert_model_weights.pth	test.cache  train.cache  val.cache


#### Load from Google drive

In [20]:
train_dataloader, val_dataloader, test_dataloader = get_dataloader('/content/RACE', '/content/drive/My Drive/Fachpraktikum/')

Loading data from /content/drive/My Drive/Fachpraktikum/train.cache...
Loading data from /content/drive/My Drive/Fachpraktikum/val.cache...
Loading data from /content/drive/My Drive/Fachpraktikum/test.cache...


In [22]:
print(len(train_dataloader))
print(len(val_dataloader))
print(len(test_dataloader))

10984
611
617


## Play around with `DataLoader`

### Get a batch of dataloader

In [19]:
sample = next(iter(test_dataloader))

### Print some information of dataloader

In [ ]:
# type of sample
print(type(sample))
# keys of sample
print(sample.keys())
# ids of sample
print(sample['id'])
# label of sample
print(sample['label'])

<class 'dict'>
dict_keys(['id', 'label', 'input_ids', 'attention_mask', 'token_type_ids'])
['middle6853.txt', 'middle6853.txt', 'middle6853.txt', 'middle5870.txt', 'middle5870.txt', 'middle5870.txt', 'middle7476.txt', 'middle7476.txt']
tensor([1, 3, 1, 0, 1, 2, 2, 2])


In [ ]:
# tokenised context and question
print(sample['input_ids'][0].size())
print(sample['input_ids'][0][0])

torch.Size([4, 128])
tensor([ 101, 2023, 2095, 1010, 1000, 3748, 2859, 1000, 2003, 3297, 1010, 2009,
        3065, 2149, 1996, 3376, 5019, 1012, 2021, 1999, 2755, 1010, 1996, 4044,
        2105, 2149, 2003, 2893, 4788, 1998, 4788, 1012, 1999, 2070, 3182, 1010,
        2057, 2064, 1005, 1056, 2156, 3869, 5742, 1999, 1996, 2314, 2030, 3628,
        2006, 1996, 4020, 1012, 2116, 4176, 2024, 5307, 1996, 5473, 1997, 2542,
        1012, 2012, 1996, 2168, 2051, 1010, 2158, 2003, 4288, 4176, 2074, 2005,
        2893, 2037, 3096, 1998, 6240, 1012, 1999, 2256, 2406, 1010, 1996, 2193,
        1997, 3748, 4176, 2003, 3352, 3760, 1998, 3760, 1012, 2070, 1997, 2068,
        2024, 2130, 5996, 2041, 1012, 2009, 1005, 1055, 2051, 2000, 4047, 2256,
        4044, 1012, 2021, 2054, 2064, 2057, 2079, 1029,  102, 2054, 2055, 2256,
        4044, 2105, 2149, 1999, 2755, 1029, 2488,  102])


### Decode tokenised context and question

In [ ]:
de_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased", do_lower_case=True)

In [ ]:
de_tokenizer.decode(sample['input_ids'][0][1])

'[CLS] this year, " wild china " is famous, it shows us the beautiful scenes. but in fact, the environment around us is getting worse and worse. in some places, we can\'t see fish swimming in the river or trees on the mountains. many animals are facing the danger of living. at the same time, man is killing animals just for getting their skin and meat. in our country, the number of wild animals is becoming smaller and smaller. some of them are even dying out. it\'s time to protect our environment. but what can we do? [SEP] what about our environment around us in fact? worse [SEP]'

## Model for training and evaluating

In [9]:
model = BertForMultipleChoice.from_pretrained("bert-base-uncased", num_labels=NUM_LABELS)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMultipleChoice: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForMultipleChoice from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMultipleChoice from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMultipleChoice were not initialized from the model checkpoint at bert-base-uncased and are newly

In [11]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# Assuming that we are on a CUDA machine, this should print a CUDA device:
print(device)

cuda:0


In [12]:
soft_max = torch.nn.Softmax(dim=1)
pred = torch.tensor([]).cuda()
i = 1
labels = torch.tensor([])
model.to(device)

for data_batch in test_dataloader:
    if i %50 == 0:
        print(str(i) + "/"+str(len(test_dataloader)))
    i = i + 1
    outputs = model(data_batch['input_ids'].cuda())
    after_softmax = soft_max(outputs.logits)
    pred = torch.cat((pred,torch.argmax(after_softmax, dim=1)))
    labels = torch.cat((labels, data_batch['label']))

In [13]:
import numpy as np
pred_np = pred.cpu().numpy().astype(np.int32)
labels_np = labels.cpu().numpy().astype(np.int32)

from sklearn.metrics import confusion_matrix, accuracy_score
print("before training, the confusion matrix is:")
print(confusion_matrix(labels_np, pred_np))
print("before training, the accuracy score is:")
print(accuracy_score(labels_np, pred_np))

before training, the confusion matrix is:
[[3 2 1 3]
 [2 6 5 4]
 [4 3 3 4]
 [3 2 2 5]]
before training, the accuracy score is:
0.3269230769230769


In [14]:
from pytorch_lightning.metrics import functional as FM
from pytorch_lightning.callbacks import ModelCheckpoint

In [15]:
class Model(pl.LightningModule):

    def __init__(self, model):
        super(Model, self).__init__()
        self.model = model

        self._train_dataloader = train_dataloader
        self._val_dataloader = val_dataloader

    def configure_optimizers(self):
        no_decay = ['bias', 'LayerNorm.weight']
        weight_decay = 0.0
        adam_epsilon = 1e-8

        optimizer_grouped_parameters = [
            {
                'params': [p for n, p in self.model.named_parameters() if not any(nd in n for nd in no_decay)],
                'weight_decay': weight_decay
                },
            {
                'params': [p for n, p in self.model.named_parameters() if any(nd in n for nd in no_decay)],
                'weight_decay': 0.0,
                }
            ]
        optimizer = AdamW(optimizer_grouped_parameters, lr=5e-5, eps=adam_epsilon)

        return optimizer

    def training_step(self, batch, batch_idx):
        labels = batch["label"]
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        token_type_ids = batch["token_type_ids"]

        outputs = self.model(
                input_ids,
                token_type_ids=token_type_ids,
                attention_mask=attention_mask,
                labels=labels
                )
        
        tqdm_dict = {"train_loss": outputs.loss}
        output = OrderedDict({
            "loss": outputs.loss,
            "progress_bar": tqdm_dict,
            "log": tqdm_dict,
            })
        
        # self.log('train_loss', outputs.loss)
        
        # self.log('train_loss', outputs.loss, on_step=True, on_epoch=True, prog_bar=True)
        return output

    def validation_step(self, batch, batch_idx):
        labels = batch["label"]
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        token_type_ids = batch["token_type_ids"]

        outputs = self.model(
                input_ids,
                token_type_ids=token_type_ids,
                attention_mask=attention_mask,
                labels=labels
                )
        labels_hat = torch.argmax(outputs.logits, dim=1)

        correct_count = torch.sum(labels == labels_hat)

        acc = FM.accuracy(labels_hat, labels)

        if self.on_gpu:
            correct_count = correct_count.cuda(outputs.loss.device.index)

        output = OrderedDict({
                "val_loss": outputs.loss,
                "accuracy": acc,
                "correct_count": correct_count,
                "batch_size": len(labels)
                })
                
        return output

    # def test_step(self, batch, batch_idx):
    #     metrics = self.validation_step(batch, batch_idx)
    #     metrics = {'test_acc': metrics['accuracy'], 'test_loss': metrics['val_loss'], "correct_count": metrics['correct_count']}
    #     self.log('test_loss', metrics['val_loss'], on_step=True, on_epoch=True, prog_bar=True)
    #     self.log('test_acc', metrics['accuracy'], on_step=True, on_epoch=True, prog_bar=True)

    def validation_end(self, outputs):
        val_acc = sum([out["correct_count"] for out in outputs]).float() / sum(out["batch_size"] for out in outputs)
        val_loss = sum([out["val_loss"] for out in outputs]) / len(outputs)
        tqdm_dict = {
                "val_loss": val_loss,
                "val_acc": val_acc,
                }
        return {"progress_bar": tqdm_dict, "log": tqdm_dict, "val_loss": val_loss}

    def train_dataloader(self):
        return self._train_dataloader

    def val_dataloader(self):
        return self._val_dataloader

In [16]:
# checkpoint_callback = ModelCheckpoint(dirpath='/content/drive/My Drive/Fachpraktikum/')
# trainer = pl.Trainer(gpus=1, max_epochs=1, callbacks=[checkpoint_callback])
trainer = pl.Trainer(gpus=1, max_epochs=2)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores


In [17]:
pl_model = Model(model)

In [18]:
trainer.fit(pl_model)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type                  | Params
------------------------------------------------
0 | model | BertForMultipleChoice | 109 M 
------------------------------------------------
109 M     Trainable params
0         Non-trainable params
109 M     Total params
437.932   Total estimated model params size (MB)


/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:68: UserWarning: The {progress_bar:dict keyword} was deprecated in 0.9.1 and will be removed in 1.0.0
Please use self.log(...) inside the lightningModule instead.
# log on a step or aggregate epoch metric to the logger and/or progress bar (inside LightningModule)
self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True)
  warnings.warn(*args, **kwargs)
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:68: UserWarning: The {log:dict keyword} was deprecated in 0.9.1 and will be removed in 1.0.0
Please use self.log(...) inside the lightningModule instead.
# log on a step or aggregate epoch metric to the logger and/or progress bar (inside LightningModule)
self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True)
  warnings.warn(*args, **kwargs)


1

In [21]:
soft_max = torch.nn.Softmax(dim=1)
pred_after_tuning = torch.tensor([]).cuda()
pl_model.model.to(device)
i = 1
for data_batch in test_dataloader:
    if i %50 == 0:
        print(str(i) + "/"+str(len(test_dataloader)))
    i = i + 1
    outputs = pl_model.model(data_batch['input_ids'].cuda())
    after_softmax = soft_max(outputs.logits)
    pred_after_tuning = torch.cat((pred_after_tuning,torch.argmax(after_softmax, dim=1)))

In [24]:
import numpy as np
pred_after_tuning_np = pred_after_tuning.cpu().numpy().astype(np.int32)


In [25]:
from sklearn.metrics import confusion_matrix
print("after fine tune")
confusion_matrix(pred_after_tuning_np, pred_np)

array([[5, 4, 3, 8],
       [5, 3, 3, 4],
       [0, 3, 1, 1],
       [2, 3, 4, 3]])

In [27]:
from sklearn.metrics import accuracy_score
accuracy_score(labels_np, pred_after_tuning_np)

0.21153846153846154

In [44]:
import numpy as np
pred_np = pred.cpu().numpy().astype(np.int32)

labels = torch.tensor([])
for data_batch in test_dataloader:
    labels = torch.cat((labels, data_batch['label']))
labels_np = labels.cpu().numpy().astype(np.int32)
from sklearn.metrics import confusion_matrix
print(confusion_matrix(labels_np, pred_np))
from sklearn.metrics import accuracy_score
print(accuracy_score(labels_np, pred_np))

[[253 250 285 275]
 [325 305 346 347]
 [330 355 281 358]
 [297 339 313 275]]
0.22578029995946494


In [29]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [32]:
torch.save(pl_model.model.state_dict(), "/content/drive/My Drive/Fachpraktikum/bert_model_weights.pth")